This dataset is downloaded from http://www2.informatik.uni-freiburg.de/~cziegler/BX/ Collected by Cai-Nicolas Ziegler in a 4-week crawl (August / September 2004) from the Book-Crossing community with kind permission from Ron Hornbaker, CTO of Humankind Systems. Contains 278,858 users (anonymized but with demographic information) providing 1,149,780 ratings (explicit / implicit) about 271,379 books.

### Popularity Based Recommendations

First we import all the libraries needed to perfrom recommendations

In [1]:
import pandas
import numpy

We have modified the original dataset slightly. You can use the this from data/books_recommendations folder. It contains Books, Users and Ratings table.

In [2]:
#As data contains non alpha numeric characters, we used latin1 encoding while loading the data
books = pandas.read_csv('./data/books_recommendation/Books.csv',encoding='latin1')
users = pandas.read_csv('./data/books_recommendation/Users.csv',sep=';',encoding='latin1')
books_ratings = pandas.read_csv('./data/books_recommendation/Book-Ratings.csv',encoding='latin1')

We can check the sample values of the tables by head() function

In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
books_ratings.head()

,rowid,User-ID,ISBN,Book-Rating
0,0,276725,034545104X,0
1,1,276726,0155061224,5
2,2,276727,0446520802,0
3,3,276729,052165615X,3
4,4,276729,0521795028,6


books_rating table contains ratings of the books whose information is not available in books table. First get the list of ISBN and remove entries whose books details are not present.

In [6]:
#Check shape of each dataframe
print('Books : ',books.shape)
print('Users : ',users.shape)
print('Books Ratings : ',books_ratings.shape)

Books :  (271379, 8)
Users :  (278858, 3)
Books Ratings :  (1149780, 4)


In [7]:
#Get all ISBN from books table
isbn = books.ISBN.unique()

In [8]:
books_ratings = books_ratings[books_ratings['ISBN'].isin(isbn)]

In [9]:
#Check the size of book_ratings dataframe
print('Books Ratings : ',books_ratings.shape)

Books Ratings :  (1031175, 4)


In [10]:
#Group by ISBN
isbn_count_list = books_ratings.groupby('ISBN')['Book-Rating'].count()
isbn_count_frame = pandas.DataFrame(isbn_count_list)

In [11]:
#Sort based on count
top_15_isbn = isbn_count_frame.sort_values('Book-Rating',ascending=False).head(n=15)

In [12]:
#Get the list of top-15 books
top_15_isbn = top_15_isbn.index.values

In [13]:
#Create a dataframe from top isbn
top_books = pandas.DataFrame(top_15_isbn,columns=['ISBN'])

In [14]:
#Fill the other books details
top_books = pandas.merge(top_books,books,on='ISBN')

In [15]:
top_books = top_books[['ISBN','Book-Title','Book-Author','Publisher']]

In [16]:
top_books

,ISBN,Book-Title,Book-Author,Publisher
0,0971880107,Wild Animus,Rich Shapero,Too Far
1,0316666343,The Lovely Bones: A Novel,Alice Sebold,Little Brown
2,0385504209,The Da Vinci Code,Dan Brown,Doubleday
3,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,Perennial
4,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,Picador USA
5,044023722X,A Painted House,John Grisham,Dell Publishing Company
6,0142001740,The Secret Life of Bees,Sue Monk Kidd,Penguin Books
7,067976402X,Snow Falling on Cedars,David Guterson,Vintage Books USA
8,0671027360,Angels &amp; Demons,Dan Brown,Pocket Star
9,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,Warner Books


In [17]:
#Check total number of authors
books['Book-Author'].describe()

count              271379
unique             102042
top       Agatha Christie
freq                  632
Name: Book-Author, dtype: object

We can see that total 102,042 unique authors are present. In top 15, John Grisham and Dan Brown appeared twice

## Content Based Recommendation Engine

We will use house dataset. This dataset is created from user responses. This is available in /data folder

<p> This dataset contains following fields</p>
<p> ID - Unique ID of each property 
<br> Size of House - Small, Medium, Large, Big
<br> Number Of Bathrooms - Number of Bathrooms available in the house
<br> Age of builing - Less than a year, 1-5 Years, 5-10 Years, 10+ Years
<br> Number of Fireplace - 0,1
<br> Furnished Status - Unfurnished, Semi Furnished, Fully Furnished
<br> Car Parking- Yes/No
<br> Air Conditioning - Yes/No
<br> Private Garden - Yes/No
<br> Life Available - Yes/No
<br> Area Status - High, Medium, Low
<br> Rent - Integer
</p>

Import libraries that is needed for this recommendation engine

In [18]:
import pandas
import numpy
import sklearn
from sklearn.neighbors import NearestNeighbors 

In [19]:
house_data = pandas.read_csv('./data/House_Survey.csv')

In [20]:
house_data.head()

,ID,Size of House,Number Of Bathrooms,Age Of Building,Number Of Balcony,Number Of Fireplace,Furnished Status,Car Parking,Air Conditioning,Private Garden,Lift Available,Area Status,Rent
0,1,Big,4,1-5 Years,3,1,Semi Furnished,Yes,Yes,Yes,No,High,50000
1,2,Small,1,Less than a year,0,0,Unfurnished,No,No,No,Yes,Medium,7000
2,3,Medium,2,5-10 Years,2,0,Fully Furnished,Yes,No,No,Yes,High,23000
3,4,Small,1,Less than a year,1,0,Semi Furnished,Yes,Yes,No,Yes,High,11500
4,5,Large,3,10+ Years,2,1,Fully Furnished,Yes,No,Yes,Yes,High,32000


To apply any algorithm first we will convert categorical variable to numeric numbers. 

We can do this with predefined functions in python. But we want to assign weights according to their categories

In [21]:
#Convert Size
house_data.loc[house_data['Size of House'] == 'Small', 'Size of House'] = 1
house_data.loc[house_data['Size of House'] == 'Medium', 'Size of House'] = 2
house_data.loc[house_data['Size of House'] == 'Large', 'Size of House'] = 3
house_data.loc[house_data['Size of House'] == 'Big', 'Size of House'] = 4

#Convert Age of Building (New buildings are more popular for rent)
house_data.loc[house_data['Age Of Building'] == 'Less than a year', 'Age Of Building'] = 4
house_data.loc[house_data['Age Of Building'] == '1-5 Years', 'Age Of Building'] = 3
house_data.loc[house_data['Age Of Building'] == '5-10 Years', 'Age Of Building'] = 2
house_data.loc[house_data['Age Of Building'] == '10+ Years', 'Age Of Building'] = 1

#Furnished Status
house_data.loc[house_data['Furnished Status'] == 'Unfurnished', 'Furnished Status'] = 1
house_data.loc[house_data['Furnished Status'] == 'Semi Furnished', 'Furnished Status'] = 1
house_data.loc[house_data['Furnished Status'] == 'Fully Furnished', 'Furnished Status'] = 1

#Car Parking
house_data.loc[house_data['Car Parking'] == 'Yes', 'Car Parking'] = 1
house_data.loc[house_data['Car Parking'] == 'No', 'Car Parking'] = 0

#Air Conditioning
house_data.loc[house_data['Air Conditioning'] == 'Yes', 'Air Conditioning'] = 1
house_data.loc[house_data['Air Conditioning'] == 'No', 'Air Conditioning'] = 0

#Private Garden
house_data.loc[house_data['Private Garden'] == 'Yes', 'Private Garden'] = 1
house_data.loc[house_data['Private Garden'] == 'No', 'Private Garden'] = 0

#Lift Available
house_data.loc[house_data['Lift Available'] == 'Yes', 'Lift Available'] = 1
house_data.loc[house_data['Lift Available'] == 'No', 'Lift Available'] = 0

#Area Status
house_data.loc[house_data['Area Status'] == 'Low', 'Area Status'] = 1
house_data.loc[house_data['Area Status'] == 'Medium', 'Area Status'] = 2
house_data.loc[house_data['Area Status'] == 'High', 'Area Status'] = 3


In [22]:
# We start by creating nearest neighbor object. 
# We use this algorithm to get the data point which is nearest to the requirement (neighbour =1)
nn1 = NearestNeighbors(n_neighbors=1)

In [23]:
#Fit the model
nn1.fit(house_data.loc[:,'Size of House':'Area Status'])

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [24]:
house_data.loc[:,'Size of House':'Area Status']

,Size of House,Number Of Bathrooms,Age Of Building,Number Of Balcony,Number Of Fireplace,Furnished Status,Car Parking,Air Conditioning,Private Garden,Lift Available,Area Status
0,4,4,3,3,1,1,1,1,1,0,3
1,1,1,4,0,0,1,0,0,0,1,2
2,2,2,2,2,0,1,1,0,0,1,3
3,1,1,4,1,0,1,1,1,0,1,3
4,3,3,1,2,1,1,1,0,1,1,3
5,2,2,4,2,0,1,1,0,0,0,1
6,1,2,3,1,1,1,1,0,0,1,2
7,4,3,4,2,1,1,1,1,1,1,3
8,3,2,3,2,0,1,1,0,0,1,3
9,1,1,2,0,0,1,1,0,0,1,2


In [25]:
#Now define our requirement in the same format (not taking ID and rent values)
requirement = [[2,2,3,1,0,2,1,0,0,0,1]]

In [26]:
#Check the most suitable house for this requirement
print(nn1.kneighbors(requirement))

(array([[ 1.73205081]]), array([[5]], dtype=int64))


In [27]:
#Checking the value in house_data at index = 5
house_data.loc[[5]]

,ID,Size of House,Number Of Bathrooms,Age Of Building,Number Of Balcony,Number Of Fireplace,Furnished Status,Car Parking,Air Conditioning,Private Garden,Lift Available,Area Status,Rent
5,6,2,2,4,2,0,1,1,0,0,0,1,11000


In [28]:
#If we want to get top-3 list
top3 = nn1.kneighbors(requirement,n_neighbors=3)[1].tolist()[0]

In [29]:
house_data.loc[top3]

,ID,Size of House,Number Of Bathrooms,Age Of Building,Number Of Balcony,Number Of Fireplace,Furnished Status,Car Parking,Air Conditioning,Private Garden,Lift Available,Area Status,Rent
5,6,2,2,4,2,0,1,1,0,0,0,1,11000
16,17,1,2,3,0,0,1,1,1,0,0,2,15600
6,7,1,2,3,1,1,1,1,0,0,1,2,12000


## Classification Based Recommendation Engine

In [30]:
#Import library
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE 

We will use bank marketing dataset in this task. This dataset is available publicaly and can be downloaded from http://archive.ics.uci.edu/ml/datasets/Bank+Marketing. 
This dataset is based on marketing calls of Portuguese banking institution.

In [31]:
#Read the bank data in pandas dataframe
bank_data = pandas.read_csv('.\\data\\bank_data.csv',sep=';')

In [32]:
#Show all columns 
pandas.set_option('display.max_columns', None)

In [33]:
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In this task we will use only few attribute to build our first classification model. 

In [34]:
bank_data = bank_data[['age','job','marital','education','default','housing','loan','contact','y']]

In [35]:
#Convert categories to 0 and 1
bank_data.y.replace(('yes', 'no'), (1, 0), inplace=True)

In [36]:
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,0
1,57,services,married,high.school,unknown,no,no,telephone,0
2,37,services,married,high.school,no,yes,no,telephone,0
3,40,admin.,married,basic.6y,no,no,no,telephone,0
4,56,services,married,high.school,no,no,yes,telephone,0


In [37]:
#Check the shape of the data
bank_data.shape

(41188, 9)

In [38]:
#Extract features form the dataset
X = bank_data.iloc[:, :7]
X = pandas.get_dummies(bank_data.iloc[:, :7]).values

#Extract class
Y = bank_data.iloc[:, 8:].values

In [39]:
#Split the data in training and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10)

In [40]:
#Create object of logistic regression
logistic_regression = LogisticRegression()

In [41]:
#Over sampling using SMOTE algorithm
smote_object = SMOTE(random_state=4)
X_res, y_res = smote_object.fit_sample(X_train, Y_train)

C:\Software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
#Fit the model
logistic_regression.fit(X_res,y_res)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
#Predict using logistic regression model
Y_pred = logistic_regression.predict(X_test)

In [44]:
#Get accuracy on test data
accuracy_score(Y_test,Y_pred)

0.5996601116775917

## Collaborative Filtering

We will use MovieLens dataset provided by grouplens for research. This dataset can be downloaded from https://grouplens.org/datasets/movielens/

In [45]:
movies = pandas.read_csv('./data/grouplens_movies.csv')
movies_ratings =  pandas.read_csv('./data/grouplens_ratings.csv')

In [46]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [47]:
movies_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [48]:
#First we check the mean rating given to each movie
movies_ratings_Data = pandas.DataFrame(movies_ratings.groupby('movieId')['rating'].mean())
movies_ratings_Data.head()

,rating
movieId,
1,3.872470
2,3.401869
3,3.161017
4,2.384615
5,3.267857


Let's see how popular each movie is among users. We can check this by adding one more column count to our movie_rating_Data dataframe

In [49]:
movies_ratings_Data['Count'] = pandas.DataFrame(movies_ratings.groupby('movieId')['rating'].count())
movies_ratings_Data.head()

,rating,Count
movieId,,
1,3.872470,247
2,3.401869,107
3,3.161017,59
4,2.384615,13
5,3.267857,56


Now we have to build user-item utility matrix. We can do this by using pivot_table() function of pandas.

In [50]:
movies_crosstab = pandas.pivot_table(data=movies_ratings, values='rating', index='userId', columns='movieId')
movies_crosstab.head()

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     4.0     NaN     NaN     NaN     NaN     NaN   

movieId  9       10      11      12      13      14      15      16      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     4.0     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     4.0     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  17      18      19      20      21      22      23      24      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           5.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  25      26      27      28      29      30      31      32      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     2.5     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  34      35      36      37      38      39      40      41      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     5.0     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           5.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     4.0     NaN     NaN   

movieId  42      43      44      45      46      47      48      49      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     4.0     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  50      52      53      54      55      57      58      59      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           4.0     3.0     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  60      61      62      63      64      65      66      68      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     3.0     NaN     NaN     NaN     NaN     NaN   
3           3.0   

Above crosstab is full of null values. This is because each user rated very less movies. Hence the above matrix is sparse.

In [51]:
#Let's take an example of movie with movieid=260
movies.loc[movies['movieId'] == 260]

,movieId,title,genres
232,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi


In [52]:
star_wars = movies_crosstab[260]

In [53]:
#It contains null values. 
star_wars = star_wars[star_wars>=0]
star_wars

userId
4      5.0
7      5.0
8      3.5
15     5.0
17     3.5
18     3.0
19     4.0
20     1.5
21     3.0
22     4.0
23     4.5
25     4.0
26     3.0
30     4.0
31     4.0
34     4.0
36     5.0
40     4.5
41     4.0
42     4.5
44     3.0
55     5.0
56     2.0
57     4.0
62     4.0
63     5.0
65     4.0
66     5.0
67     4.0
68     3.0
      ... 
597    5.0
598    5.0
599    5.0
602    4.0
603    5.0
607    4.5
611    2.0
613    5.0
614    4.0
615    3.5
616    5.0
620    4.0
622    5.0
624    5.0
626    3.0
627    4.0
636    3.0
640    5.0
643    5.0
646    5.0
647    5.0
651    5.0
654    5.0
655    2.5
659    5.0
660    4.5
664    4.5
665    4.0
669    5.0
671    5.0
Name: 260, Length: 291, dtype: float64

Now our objective is to find similar movies. We will understand this by finding movies similar to star wars

In [54]:
similar = movies_crosstab.corrwith(star_wars)
similar_dframe = pandas.DataFrame(similar,columns=['PearsonR'])

C:\Software\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Software\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


In [55]:
#Drop na values
similar_dframe.dropna(inplace=True)
similar_dframe.head()

,PearsonR
movieId,
1,0.281415
2,0.143204
3,0.271254
4,-0.385922
5,0.280379


Above shows each movie id and their correlation with star_wars. This is not very useful at this moment.

We also want to check for popular movies only which are rated by atleast 50 users. To do this let's join our frame with rating.

In [56]:
similar_dframe_summary = similar_dframe.join(movies_ratings_Data['Count'])
similar_dframe_summary = similar_dframe_summary[similar_dframe_summary['Count']>=50].sort_values('PearsonR', ascending=False).head(5)

In [57]:
top_5_similar = similar_dframe_summary.head(5).index.values.tolist()

In [58]:
#Print top 10 similar movies to Star Wars
movies.loc[movies['movieId'].isin(top_5_similar)]

,movieId,title,genres
232,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
953,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
966,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
6944,59315,Iron Man (2008),Action|Adventure|Sci-Fi
7218,68358,Star Trek (2009),Action|Adventure|Sci-Fi|IMAX



Above result is very interesting. First record is the movie itself. But second and third are the next part of similar movies. So, if a user likes Star Wars IV, we can recommend him another two movies in the same series. These results are self-explainable. This is also a reason why collaborative filtering method are very popular in recommendation engine techniques.
